In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [16]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [17]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [18]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [24]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0312, -0.0175,  0.0385,  0.0239, -0.0763, -0.0385,  0.0765,  0.0755,
          0.0980,  0.0656]], grad_fn=<ThAddmmBackward>)


In [25]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [26]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(1.1861, grad_fn=<MseLossBackward>)


In [27]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [28]:
net.zero_grad()
print('conv1.bias.grad before')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after')
print(net.conv1.bias.grad)

conv1.bias.grad before
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after
tensor([-0.0035, -0.0099,  0.0089,  0.0019, -0.0199, -0.0101])


In [29]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [31]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [32]:
# In training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()